In [1]:
import numpy as np
from scipy.optimize import fsolve, fixed_point
from matplotlib import pyplot as plt
import pyblp
from tqdm.notebook import trange
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

In [2]:
RNG_SEED = 8476263

rng = np.random.default_rng(RNG_SEED) # this random seeding is for reproducibility

In [3]:
# I am horrified that we have to overrun the default collinearity checks
# however, wired and satellite dummy variables are collinear
# so to prevent PyBLP from throwing a fit, we must do this.
pyblp.options.collinear_rtol = 0
pyblp.options.collinear_atol = 0

In [4]:
# fixed parameter definitions

beta1 = 1
alpha = -2
gamma0 = 1/2
gamma1 = 1/4
beta2_bar = 4
beta3_bar = 4
sigma2 = 1
sigma3 = 1

# markets and goods
T = 600
J = 4

In [5]:
# 3.1

# x_jt, w_jt are absolute value of iid standard normal variables
x = np.absolute(rng.standard_normal(size=(J,T)))
w = np.absolute(rng.standard_normal(size=(J,T)))

unobservable_mean = [0,0]
unobservable_cov = [[1,0.25],[0.25,1]]
unobservables = rng.multivariate_normal(unobservable_mean, unobservable_cov, size=(J,T))
xi = unobservables[:,:,0]
omega = unobservables[:,:,1]

In [6]:
# 3.2a
# defining the market share

def own_mkt_share_derivative(t, p, beta2, beta3): 
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T
    numerator = alpha*np.exp(u_t)*Z - alpha*np.square(np.exp(u_t)) # num_sims x J
    denominator = np.square(Z)
    
    return np.mean(numerator / denominator, axis=0)

    
    
def full_mkt_share_derivative(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T # num_sims x J
    
    derivatives = np.zeros((J,J))
    
    own_numerator = alpha*np.exp(u_t)*Z - alpha*np.square(np.exp(u_t)) # num_sims x J
    denominator = np.square(Z)
    
    for j in range(J):
        derivatives[j,j] = np.mean(own_numerator / denominator, axis=0)[j]
        
    for j in range(J):
        for k in range(J):
            if not (j == k):
                derivatives[j,k] = np.mean(-1*alpha*np.exp(u_t)[:,k]*np.exp(u_t)[:,j] / np.square(1 + np.sum(np.exp(u_t),axis=-1)))
    
    return derivatives

In [7]:
# s_jt(p) 
def mkt_share(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    numerator = np.exp(u_t) 
    denominator = 1 + np.sum(np.exp(u_t),axis=-1) # num_sims
    
    return np.mean(numerator / (np.tile(denominator, (J, 1)).T), axis=0) 


In [8]:
# 3.2a(iv)
# draw beta coefficients for N individuals S times, observe variation in market share derivatives

S = 100

all_derivatives = np.zeros((J,J,S))
all_shares = np.zeros((J,S))

N = 3000

price = np.array([1,1,1,1])

for s in trange(S):
    beta2 = rng.normal(beta2_bar, sigma2, N)
    beta3 = rng.normal(beta3_bar, sigma3, N)
    all_derivatives[:,:,s] = full_mkt_share_derivative(0, price, beta2, beta3)
    all_shares[:,s] = mkt_share(1, price, beta2, beta3)
(np.mean(all_shares,axis=1), np.std(all_shares,axis=1), np.mean(all_derivatives, axis=2), np.std(all_derivatives, axis=2))

(array([0.04784253, 0.15288083, 0.44046486, 0.35277411]),
 array([0.0008991 , 0.00287306, 0.00211771, 0.0016961 ]),
 array([[-0.29478105,  0.06650959,  0.2168944 ,  0.00709914],
        [ 0.06650959, -0.16315672,  0.09183023,  0.00300568],
        [ 0.2168944 ,  0.09183023, -0.35012373,  0.03100105],
        [ 0.00709914,  0.00300568,  0.03100105, -0.04144621]]),
 array([[3.08401066e-03, 1.64034900e-03, 1.89106999e-03, 6.18963701e-05],
        [1.64034900e-03, 2.14278030e-03, 8.00654110e-04, 2.62061074e-05],
        [1.89106999e-03, 8.00654110e-04, 2.45783477e-03, 3.51894072e-04],
        [6.18963701e-05, 2.62061074e-05, 3.51894072e-04, 2.89493485e-04]]))

In [9]:
mc = np.exp( gamma0 + gamma1*w + omega/8)

In [10]:
# define function to solve

def get_function_to_solve(t, beta2, beta3):
    def F(p):
        # p is a 
        ds_dp = own_mkt_share_derivative(t, p, beta2, beta3)
        shares = mkt_share(t, p, beta2, beta3)
        return p - mc[:,t] + np.reciprocal(ds_dp)*shares
        
    return F


In [11]:
# draw betas, now compute equilibrium prices and shares

beta2 = rng.normal(beta2_bar, sigma2, N)
beta3 = rng.normal(beta3_bar, sigma3, N)

In [12]:


# 3.2 and 3.3: compute equilibrium shares, prices

# these two variables are the prices and shares
eq_prices = np.zeros((J, T))
eq_shares = np.zeros((J, T))

flag_total = 0

for t in trange(T):
    fn = get_function_to_solve(t, beta2, beta3)
    mkt_eq_prices, _ , flag, _ = fsolve(fn, np.array([1,1,1,1]), full_output=True)
    flag_total += flag
    eq_prices[:,t] = mkt_eq_prices
    eq_shares[:, t] = mkt_share(t, mkt_eq_prices, beta2, beta3)
    
# this should be True iff all of the fsolves converge
flag_total == T

True

In [ ]:
# TODO: check that at the equilibrium prices, the estimates for market shares and market share derivatives are precise
#repeating the exercise of simulation with equilibrium prices, trying to get equilibrium shares

S = 100

all_derivatives = np.zeros((J,J,S))
all_shares = np.zeros((J,S))

N = 100

for t in range(T):
    price = np.array(eq_prices[:,t])
    for s in range(S):
        beta2_s = np.random.normal(beta2_bar, sigma2, N)
        beta3_s = np.random.normal(beta3_bar, sigma3, N)
        all_derivatives[:,:,s] = full_mkt_share_derivative(0, price, beta2_s, beta3_s)
        all_shares[:,s] = mkt_share(1, price, beta2_s, beta3_s)
    
(np.mean(all_shares,axis=1), np.std(all_shares,axis=1), np.mean(all_derivatives, axis=2), np.std(all_derivatives, axis=2))

In [ ]:
# Morrow and Skerlos (2011) Method: (see equation 27 in Conlon + Gortmaker)

def get_matrices(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T # num_sims x J
    
    Lambda_inv = np.zeros((J,J))
    Gamma = np.zeros((J,J))
    
    own_numerator = alpha*np.exp(u_t)  # num_sims x J
    denominator = Z
    
    for j in range(J):
        Lambda_inv[j,j] = 1 / (np.mean(own_numerator / denominator, axis=0)[j])
        
    for j in range(J):
        for k in range(J):
            Gamma[j,k] = np.mean(alpha*np.exp(u_t)[:,k]*np.exp(u_t)[:,j] / np.square(1 + np.sum(np.exp(u_t),axis=-1)))
    
    return Lambda_inv, Gamma

def get_fixed_point_function(t, beta2, beta3):
    ownership_matrix = np.identity(J)
    def F(p):
        Lambda_inv, Gamma = get_matrices(t, p, beta2, beta3)
        shares = mkt_share(t, p, beta2, beta3)
        zeta = np.matmul(np.matmul(Lambda_inv, ownership_matrix*Gamma), (p - mc[:,t])) - np.matmul(Lambda_inv, shares)
        return mc[:,t] + zeta
        
    return F



In [ ]:
# Simulate equilibrium using the Morrow and Skerlos (2011) method
eq_prices_2 = np.zeros((J, T))
eq_shares_2 = np.zeros((J, T))

for t in trange(T):
    fn = get_fixed_point_function(t, beta2, beta3)
    mkt_eq_prices = fixed_point(fn, np.array([1,1,1,1]), method="iteration")
    eq_prices_2[:,t] = mkt_eq_prices
    eq_shares_2[:, t] = mkt_share(t, mkt_eq_prices, beta2, beta3)

# the difference between the two methods, check that this is small
np.max(eq_prices_2 - eq_prices), np.max(eq_shares_2 - eq_shares)

In [ ]:
# Pick the results from the fsolve method (slightly faster)
market_ids = np.tile(np.arange(T) + 1,(J,1)).T.flatten()
firm_ids = np.tile(np.arange(J) + 1,(T,1)).flatten()
satellite = np.concatenate((np.ones((2,T)), np.zeros((2,T)))).T.flatten()
wired = np.concatenate((np.zeros((2,T)), np.ones((2,T)))).T.flatten()
observed_data = pd.DataFrame(data={
    "market_ids": market_ids, 
    "firm_ids": firm_ids,
    "shares": eq_shares.T.flatten(), 
    "prices": eq_prices.T.flatten(),
    "x": x.T.flatten(),
    "satellite": satellite,
    "wired": wired,
    "w": w.T.flatten()
})
unobserved_data = pd.DataFrame(data={
    "market_ids": market_ids, 
    "firm_ids": firm_ids,
    "xi": xi.T.flatten(),
    "omega": omega.T.flatten()
})

In [ ]:
# Instrument Analysis

df1 = pd.DataFrame({'p':eq_prices[0,:]})
df1['s'] = pd.Series(eq_shares[0,:])
df1['x'] = pd.Series(x[0,:])
df1['w'] = pd.Series(w[0,:])

df2 = pd.DataFrame({'p':eq_prices[1,:]})
df2['s'] = pd.Series(eq_shares[1,:])
df2['x'] = pd.Series(x[1,:])
df2['w'] = pd.Series(w[1,:])

df3 = pd.DataFrame({'p':eq_prices[2,:]})
df3['s'] = pd.Series(eq_shares[2,:])
df3['x'] = pd.Series(x[2,:])
df3['w'] = pd.Series(w[2,:])


df4 = pd.DataFrame({'p':eq_prices[3,:]})
df4['s'] = pd.Series(eq_shares[3,:])
df4['x'] = pd.Series(x[3,:])
df4['w'] = pd.Series(w[3,:])


modelp11 = smf.ols('p ~ x +w', data=df1)
modelp11 = modelp11.fit()

modelp21 = smf.ols('p ~ x +w', data=df2)
modelp21 = modelp21.fit()

modelp31 = smf.ols('p ~ x +w', data=df3)
modelp31 = modelp31.fit()

modelp41 = smf.ols('p ~ x +w', data=df4)
modelp41 = modelp41.fit()


models11 = smf.ols('s ~ x +w', data=df1)
models11 = models11.fit()

models21 = smf.ols('s ~ x +w', data=df2)
models21 = models21.fit()

models31 = smf.ols('s ~ x +w', data=df3)
models31 = models31.fit()

models41 = smf.ols('s ~ x +w', data=df4)
models41 = models41.fit()

In [ ]:
modelp11.summary()

## Part 4

In [ ]:
# 4A: Logit
outside_shares = 1 - np.sum(eq_shares, axis=0, keepdims=True)
y = np.log(eq_shares/outside_shares).T.flatten()
X = observed_data[["x","satellite","wired","prices"]]
results = sm.OLS(y,X).fit()
results.summary()

Note that ignoring the endogeneity of prices results in underestimating the magnitudes of all the relevant parameters.

## Part 5

In [ ]:
# BLP, Demand-side estimation only

pyblp_problem = pyblp.Problem(
    [
        pyblp.Formulation("0 + prices + x + satellite + wired"),
        pyblp.Formulation("0 + satellite + wired"),
        None
    ],
    observed_data,
    integration=pyblp.Integration('product', size=5),
)

In [ ]:
sim = pyblp.Simulation(
    product_formulations=[
        pyblp.Formulation("0 + prices + x + satellite + wired"),
        pyblp.Formulation("0 + satellite + wired"),
        pyblp.Formulation("1 + w")
    ],
    beta=[alpha, beta1, beta2_bar, beta3_bar],
    sigma=[[sigma2, 0],[0, sigma3]],
    gamma=[gamma0, gamma1],
    product_data = observed_data[["market_ids","firm_ids","x","satellite","wired","w"]],
    integration=pyblp.Integration('product', size=9),
    xi = unobserved_data.xi,
    omega = unobserved_data.omega/8,
    costs_type="log"
)